In [1]:
import os 

from dotenv import load_dotenv
load_dotenv()


OLLAMA_URL=os.getenv("OLLAMA_URL")
LLM_MODEL=os.getenv("LLM_MODEL")
EMBEDDING_MODEL=os.getenv("EMBEDDING_MODEL")
DATAPATH=os.getenv("DATAPATH")
#OLLAMA_URL='http://localhost:11434'
#LLM_MODEL="llama3.2:1b"
LLM_MODEL="llama3.2:3b" # max retires
#LLM_MODEL="gemma3:4b" # no tool support
#LLM_MODEL="llava:7b"
#LLM_MODEL="codellama:7b"
#LLM_MODEL="qwen2.5-coder:1.5b" # no tool support
#LLM_MODEL="deepseek-r1:1.5b" # does not support tools
#os.setenv("no_proxy=localhost,127.0.0.1,*.my-it-solutions.net,ollama")

print(OLLAMA_URL)
print(f"model: {LLM_MODEL}")
print(f"embedding: {EMBEDDING_MODEL}")
print(f"datapath {DATAPATH}")
print(f"no_proxy {os.getenv('no_proxy')}")

import requests
import time

response = requests.get(OLLAMA_URL,timeout=3)
if response.status_code == requests.codes.ok:
    print(response)
else:
    print(response)    

http://ollama-service-internal.default.svc.cluster.local:11433
model: llama3.2:3b
embedding: bge-m3:567m
datapath /app/data
no_proxy None
<Response [200]>


In [2]:
#from llama_index.llms.ollama import Ollama
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
#llm = Ollama(model=LLM_MODEL, request_timeout=120.0, base_url=OLLAMA_URL, temperature=0.0)


ollama_model = OpenAIModel(
    model_name=LLM_MODEL, provider=OpenAIProvider(base_url=f"{OLLAMA_URL}/v1")
)


In [3]:
from mcp.server.fastmcp import FastMCP

from starlette.applications import Starlette
from starlette.routing import Mount, Host
import uvicorn

from pydantic_ai import Agent
import asyncio
import nest_asyncio
nest_asyncio.apply()
#loop = asyncio.get_event_loop()


server = FastMCP('PydanticAI Server')
server_agent = Agent(model=ollama_model,
              result_type=str,
              deps_type=str,
              system_prompt="",              
              retries=2)

@server.tool()
async def poet(theme: str) -> str:
    """Poem generator"""
    print("test")
    r = await server_agent.run(f'write a poem about {theme}')    
    return r.data

@server.prompt()
def review_code(code: str) -> str:
    return f"Please review this code:\n\n{code}"    


#server.run()
#asyncio.run_coroutine_threadsafe(server.run('sse'), loop)
#asyncio.run(server.run('sse'))
# Mount the SSE server to the existing ASGI server
app = Starlette(
    routes=[
        Mount('/', app=server.sse_app()),
    ]
)

# or dynamically mount as host
app.router.routes.append(Host('localhost', app=server.sse_app()))

#LLM_MODEL="llama3.2:3b.run(app, host="127.0.0.1", port=8000)
bg = None
async def start_mcp_server(bg):
    bg = await uvicorn.run(app, host='0.0.0.0', port=8001, log_level="info")
    #config = uvicorn.Config("notebook:app", host='0.0.0.0', port=8001, log_level="info")
    #server = uvicorn.Server(config)
    #await server.serve()

if bg is not None:
    print("alrady runnign, stop")
    bg.stop()

asyncio.create_task(start_mcp_server(bg))
#start_mcp_server(bg)
print("started")

started


INFO:     Started server process [2151]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


INFO:     127.0.0.1:57388 - "GET /sse HTTP/1.1" 200 OK
INFO:     127.0.0.1:57392 - "POST /messages/?session_id=fba5ebeee4ad43c1b86bb17244cb8e54 HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:57392 - "POST /messages/?session_id=fba5ebeee4ad43c1b86bb17244cb8e54 HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:57392 - "POST /messages/?session_id=fba5ebeee4ad43c1b86bb17244cb8e54 HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:57910 - "GET /sse HTTP/1.1" 200 OK
INFO:     127.0.0.1:57924 - "POST /messages/?session_id=0d948018761541bdaff5a90163a5969c HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:57924 - "POST /messages/?session_id=0d948018761541bdaff5a90163a5969c HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:57924 - "POST /messages/?session_id=0d948018761541bdaff5a90163a5969c HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:57924 - "POST /messages/?session_id=0d948018761541bdaff5a90163a5969c HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:57924 - "POST /messages/?session_id=0d948018761541bdaff5a90163a5969c HTTP/1.1" 202 Acc

In [4]:
print("test")

test


In [6]:
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerHTTP

server = MCPServerHTTP(url='http://localhost:8001/sse')  
agent = Agent(model=ollama_model,
              result_type=str,
              deps_type=str,
              system_prompt="",
              mcp_servers=[server],
              retries=1)


async def main():
    async with agent.run_mcp_servers():  
        result = await agent.run('Schreibe ein Gedicht über Füsse')
    print(result.data)
    #> There are 9,208 days between January 1, 2000, and March 18, 2025.

result = await main()
print(result)

[03/27/25 23:45:58] INFO     Connecting to SSE endpoint: http://localhost:8001/sse                        ]8;id=594244;file:///opt/venv/lib/python3.13/site-packages/mcp/client/sse.py\sse.py]8;;\:]8;id=908773;file:///opt/venv/lib/python3.13/site-packages/mcp/client/sse.py#45\45]8;;\

                    INFO     HTTP Request: GET http://localhost:8001/sse "HTTP/1.1 200 OK"          ]8;id=612761;file:///opt/venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=31349;file:///opt/venv/lib/python3.13/site-packages/httpx/_client.py#1786\1786]8;;\

                    INFO     Received endpoint URL:                                                       ]8;id=657523;file:///opt/venv/lib/python3.13/site-packages/mcp/client/sse.py\sse.py]8;;\:]8;id=115673;file:///opt/venv/lib/python3.13/site-packages/mcp/client/sse.py#65\65]8;;\
                             http://localhost:8001/messages/?session_id=0d948018761541bdaff5a90163a5969c           

                    INFO     Starting post writer with endpoint URL:                                     ]8;id=210838;file:///opt/venv/lib/python3.13/site-packages/mcp/client/sse.py\sse.py]8;;\:]8;id=222778;file:///opt/venv/lib/python3.13/site-packages/mcp/client/sse.py#135\135]8;;\
                             http://localhost:8001/messages/?session_id=0d948018761541bdaff5a90163a5969c           

                    INFO     HTTP Request: POST                                                     ]8;id=301562;file:///opt/venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=617665;file:///opt/venv/lib/python3.13/site-packages/httpx/_client.py#1786\1786]8;;\
                             http://localhost:8001/messages/?session_id=0d948018761541bdaff5a90163a                
                             5969c "HTTP/1.1 202 Accepted"                                                         

...

[03/27/25 23:46:04] INFO     HTTP Request: POST                                                     ]8;id=807432;file:///opt/venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=579973;file:///opt/venv/lib/python3.13/site-packages/httpx/_client.py#1786\1786]8;;\
                             http://ollama-service-internal.default.svc.cluster.local:11433/v1/chat                
                             /completions "HTTP/1.1 200 OK"                                                        

                    INFO     HTTP Request: POST                                                     ]8;id=719310;file:///opt/venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=978043;file:///opt/venv/lib/python3.13/site-packages/httpx/_client.py#1786\1786]8;;\
                             http://localhost:8001/messages/?session_id=0d948018761541bdaff5a90163a                
                             5969c "HTTP/1.1 202 Accepted"                                                         

                    INFO     Processing request of type ListToolsRequest                              ]8;id=355600;file:///opt/venv/lib/python3.13/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=134556;file:///opt/venv/lib/python3.13/site-packages/mcp/server/lowlevel/server.py#534\534]8;;\

[03/27/25 23:46:13] INFO     HTTP Request: POST                                                     ]8;id=576840;file:///opt/venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=683789;file:///opt/venv/lib/python3.13/site-packages/httpx/_client.py#1786\1786]8;;\
                             http://ollama-service-internal.default.svc.cluster.local:11433/v1/chat                
                             /completions "HTTP/1.1 200 OK"                                                        

Das Gedicht behandelt die komplexe und vielschichtige Natur des Wortes "Füße". Es beschreibt wie es oft als abwertend oder schmähend angesehen wird, aber auch eine Rolle bei sexueller Lust und Verführung spielen kann. Das Gedicht spielt auch mit der Idee, dass Füsse oft als "geliebter Feind" wahrgenommen werden, da sie sowohl kultiviert als auch rauen Canalisierungen für Intimität bieten können.

Das Gedicht nimmt die Leser auf eine Reise durchs verschiedene Gefühle und Assoziationen mit Füßen führt, von verbotener Lust bis hin zu Perversion. Es betont dabei, dass Füße oft ignoriert werden können, aber auch eine Rolle bei der Intimität und im Verhältnis zwischen zwei Lebewesen spielen können.

Durch die Verwendung von Metaphern und Symbolik schafft das Gedicht ein atmosphärisches und tiefes Umfeld zum Erforschen der komplexen Natur des Füßes. Es ist eine Aufforderung, über die Bedeutung und Signifikanz der Füße nachzudenken und sie weder zu ignorieren noch als Macht loszuwerden.
None
